In [1]:
import os
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose
from tensorflow.keras import Sequential
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.saving import load_model
from skimage.morphology import disk

In [2]:
seg_train = sorted(glob(os.path.join('.\Train\Train','*.png')))
print(len(seg_train))

1945


In [3]:
seg_test = sorted(glob(os.path.join('.\Test\Test','*.png')))
print(len(seg_test))

648


In [4]:
print(seg_train[:5])

['.\\Train\\Train\\ISIC_0000000_seg.png', '.\\Train\\Train\\ISIC_0000001_seg.png', '.\\Train\\Train\\ISIC_0000003_seg.png', '.\\Train\\Train\\ISIC_0000004_seg.png', '.\\Train\\Train\\ISIC_0000007_seg.png']


In [5]:
print(seg_test[:5])

['.\\Test\\Test\\ISIC_0000006_seg.png', '.\\Test\\Test\\ISIC_0000011_seg.png', '.\\Test\\Test\\ISIC_0000014_seg.png', '.\\Test\\Test\\ISIC_0000018_seg.png', '.\\Test\\Test\\ISIC_0000022_seg.png']


In [6]:
train_ims = []
for i in range(len(seg_train)):
    name = seg_train[i]
    name = name.replace('_seg.png','.jpg')
    name = name.replace('.\\Train\\Train\\','./Train/Modified/')
    train_ims.append(name)

In [7]:
test_ims = []
for i in range(len(seg_test)):
    name = seg_test[i]
    name = name.replace('_seg.png','.jpg')
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    test_ims.append(name)

In [8]:
print(test_ims[:5])

['./Test/Modified/ISIC_0000006.jpg', './Test/Modified/ISIC_0000011.jpg', './Test/Modified/ISIC_0000014.jpg', './Test/Modified/ISIC_0000018.jpg', './Test/Modified/ISIC_0000022.jpg']


In [9]:
print(train_ims[:5])

['./Train/Modified/ISIC_0000000.jpg', './Train/Modified/ISIC_0000001.jpg', './Train/Modified/ISIC_0000003.jpg', './Train/Modified/ISIC_0000004.jpg', './Train/Modified/ISIC_0000007.jpg']


In [10]:
in_layer = Input((384,512,1), name='input')
conv1 = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(in_layer)
conv2 = Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)(conv1)
conv3 = Conv2D(4, (3, 3), activation='relu', padding='same', strides=2)(conv2)
conv4 = Conv2DTranspose(4, kernel_size=3, strides=2, activation='relu', padding='same')(conv3)
conv5 = Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same')(conv4)
conv6 = Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same')(conv5)
out_layer = Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')(conv6)

autoencoder = keras.Model(in_layer,out_layer)
autoencoder.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 384, 512, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 192, 256, 16)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 128, 8)        1160      
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 64, 4)         292       
                                                                 
 conv2d_transpose (Conv2DTr  (None, 96, 128, 4)        148       
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 192, 256, 8)       296       
 Transpose)                                                 

In [11]:
autoencoder.compile(optimizer='adam', loss=MeanSquaredError())

In [12]:
train_ims = np.array([cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384))/255 for x in train_ims])
train_masks = np.array([cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384))/255 for x in seg_train])

test_ims = np.array([cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384))/255 for x in test_ims])
test_masks = np.array([cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384))/255 for x in seg_test])

In [13]:
print(train_ims.shape)
print(test_ims.shape)
print(train_masks.shape)
print(test_masks.shape)

(1945, 384, 512)
(648, 384, 512)
(1945, 384, 512)
(648, 384, 512)


In [14]:
data = train_ims.reshape((1945,384,512,1))
masks = train_masks.reshape((1945,384,512,1))

In [15]:
test_data = test_ims.reshape((648,384,512,1))
test_masks = test_masks.reshape((648,384,512,1))

In [17]:
h = autoencoder.fit(data,masks,epochs=10,batch_size=32,verbose=1,shuffle=True,validation_data=(test_data,test_masks))

Epoch 1/10

61/61 [==============================] - 457s 7s/step - loss: 0.1972 - val_loss: 0.1597
Epoch 2/10
61/61 [==============================] - 248s 4s/step - loss: 0.1550 - val_loss: 0.1483
Epoch 3/10
61/61 [==============================] - 342s 6s/step - loss: 0.1426 - val_loss: 0.1359
Epoch 4/10
61/61 [==============================] - 184s 3s/step - loss: 0.1364 - val_loss: 0.1333
Epoch 5/10
61/61 [==============================] - 160s 3s/step - loss: 0.1359 - val_loss: 0.1343
Epoch 6/10
61/61 [==============================] - 148s 2s/step - loss: 0.1348 - val_loss: 0.1317
Epoch 7/10
61/61 [==============================] - 151s 2s/step - loss: 0.1340 - val_loss: 0.1307
Epoch 8/10
61/61 [==============================] - 142s 2s/step - loss: 0.1333 - val_loss: 0.1299
Epoch 9/10
61/61 [==============================] - 134s 2s/step - loss: 0.1320 - val_loss: 0.1291
Epoch 10/10
61/61 [==============================] - 149s 2s/step - loss: 0.1319 - val_loss: 0.1285


In [18]:
autoencoder.save('autoencoder_seg.keras')

In [19]:
autoencoder = load_model("autoencoder_seg.keras")
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 384, 512, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 192, 256, 16)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 128, 8)        1160      
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 64, 4)         292       
                                                                 
 conv2d_transpose (Conv2DTr  (None, 96, 128, 4)        148       
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 192, 256, 8)       296       
 Transpose)                                                  